In [1]:
import torchaudio
torchaudio.set_audio_backend("soundfile")

from datasets.Sampler import Sampler
from graphs.models.SpeakerNet import SpeakerNet
from graphs.models.VGGVox import VGGVox

from agents.trainer import Trainer


from utils.config import *

#JSON = 'configs/VGGVox.json'
#JSON = 'configs/ResNetSE34L.json'
JSON = 'configs/Test.json'

config = process_config(JSON)

C:\Users\pagli\Anaconda3\lib\site-packages\torchaudio\extension\extension.py:14: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\pagli\Anaconda3\lib\site-packages\torchaudio\backend\utils.py:64: UserWarning: The interface of "soundfile" backend is planned to change in 0.8.0 to match that of "sox_io" backend and the current interface will be removed in 0.9.0. To use the new interface, do `torchaudio.USE_SOUNDFILE_LEGACY_INTERFACE = False` before setting the backend to "soundfile". Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  'The interface of "soundfile" backend is planned to change in 0.8.0 to '
[INFO]: Experiment: VGGVox.


In [2]:
trainer = Trainer(config)

[INFO]: Operation will be on ***** CPU ***** 


ModuleAttributeError: 'VGGVox' object has no attribute 'module'

In [ ]:
trainer.run()